In [1]:
import boto3
import uuid
s3 = boto3.resource('s3', region_name='us-west-1')

In [2]:
bucket_name = 'yahoofinancestockprice-' + str(uuid.uuid4())
try:
    s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint':'us-west-1'})
    print('S3 bucket has been created successfully')
except Exception as e:
    print('S3 error: ', e)

S3 bucket has been created successfully


In [3]:
!pip install yfinance
import pandas as pd
from datetime import datetime
import yfinance as yf

#initialize parameter
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 1, 1)

#get the data
df_data = yf.download('AAPL', start = start_date, end = end_date)

df_data.reset_index(inplace=True)

df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-03,177.830002,182.880005,177.710007,182.009995,179.481110,104487900
1,2022-01-04,182.630005,182.940002,179.119995,179.699997,177.203232,99310400
2,2022-01-05,179.610001,180.169998,174.639999,174.919998,172.489594,94537600
3,2022-01-06,172.699997,175.300003,171.639999,172.000000,169.610214,96904000
4,2022-01-07,172.889999,174.139999,171.029999,172.169998,169.777832,86709100
...,...,...,...,...,...,...,...
496,2023-12-22,195.179993,195.410004,192.970001,193.600006,193.091385,37122800
497,2023-12-26,193.610001,193.889999,192.830002,193.050003,192.542831,28919300
498,2023-12-27,192.490005,193.500000,191.089996,193.149994,192.642548,48087700
499,2023-12-28,194.139999,194.660004,193.169998,193.580002,193.071426,34049900


In [4]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace=True)

In [5]:
df_data_features = df_data.iloc[:-1 , :]
df_data_features

,Open,High,Low,Close,Volume
0,177.830002,182.880005,177.710007,182.009995,104487900
1,182.630005,182.940002,179.119995,179.699997,99310400
2,179.610001,180.169998,174.639999,174.919998,94537600
3,172.699997,175.300003,171.639999,172.000000,96904000
4,172.889999,174.139999,171.029999,172.169998,86709100
...,...,...,...,...,...
495,196.100006,197.080002,193.500000,194.679993,46482500
496,195.179993,195.410004,192.970001,193.600006,37122800
497,193.610001,193.889999,192.830002,193.050003,28919300
498,192.490005,193.500000,191.089996,193.149994,48087700


In [6]:
df_data_targets = df_data.iloc[1:, 0].rename("Targets")
df_data_targets

1      182.630005
2      179.610001
3      172.699997
4      172.889999
5      169.080002
          ...    
496    195.179993
497    193.610001
498    192.490005
499    194.139999
500    193.899994
Name: Targets, Length: 500, dtype: float64

In [7]:
df_data_features['Target'] = list(df_data_targets)

first_column = df_data_features.pop('Target')
df_data_features.insert(0, 'Target', first_column)

df_data_final = df_data_features
df_data_final

/tmp/ipykernel_1982/2375105290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features['Target'] = list(df_data_targets)


,Target,Open,High,Low,Close,Volume
0,182.630005,177.830002,182.880005,177.710007,182.009995,104487900
1,179.610001,182.630005,182.940002,179.119995,179.699997,99310400
2,172.699997,179.610001,180.169998,174.639999,174.919998,94537600
3,172.889999,172.699997,175.300003,171.639999,172.000000,96904000
4,169.080002,172.889999,174.139999,171.029999,172.169998,86709100
...,...,...,...,...,...,...
495,195.179993,196.100006,197.080002,193.500000,194.679993,46482500
496,193.610001,195.179993,195.410004,192.970001,193.600006,37122800
497,192.490005,193.610001,193.889999,192.830002,193.050003,28919300
498,194.139999,192.490005,193.500000,191.089996,193.149994,48087700


In [8]:
import numpy as np

df_randomized = df_data_final.sample(frac=1, random_state=123)
df_randomized

,Target,Open,High,Low,Close,Volume
229,148.210007,141.399994,148.720001,140.550003,148.029999,111380900
337,173.050003,172.479996,173.850006,172.110001,173.500000,55962800
327,165.190002,165.000000,165.600006,163.889999,165.330002,41949600
416,187.839996,184.940002,187.850006,184.740005,187.649994,60813900
306,158.860001,158.830002,161.550003,157.679993,158.929993,67622100
...,...,...,...,...,...,...
98,138.429993,140.809998,141.970001,137.330002,140.360001,104132700
476,189.919998,190.869995,190.899994,189.250000,189.970001,24048300
322,166.100006,165.089996,165.389999,164.029999,165.229996,41516200
382,190.229996,190.500000,191.190002,189.779999,190.539993,41342300


In [9]:
train_data, test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])

print(train_data.shape, test_data.shape)

(400, 6) (100, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [10]:
import os

prefix = 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train', 'train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test', 'test.csv')

print(train_csv_path)
print(test_csv_path)

s3://yahoofinancestockprice-be021548-edd3-40c5-a8fa-f96e9497d6c8/xgboost-as-a-built-in-algo/train/train.csv
s3://yahoofinancestockprice-be021548-edd3-40c5-a8fa-f96e9497d6c8/xgboost-as-a-built-in-algo/test/test.csv


In [ ]:
Reference: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [12]:
train_data.to_csv(train_csv_path, index = False, header = False)
test_data.to_csv(test_csv_path, index = False, header = False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [14]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [15]:
xgboost_container = image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-2")

display(xgboost_container)

'746614075791.dkr.ecr.us-west-1.amazonaws.com/sagemaker-xgboost:1.2-2'

In [16]:
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"reg:squarederror",
    "early_stopping_rounds":10,
    "num_round":1000
}

In [17]:
# first {} is bucket name
# second {} is prefix
# output folder
output_path = 's3://{}/{}/{}/'.format(bucket_name, prefix, 'output')

print(output_path)

s3://yahoofinancestockprice-be021548-edd3-40c5-a8fa-f96e9497d6c8/xgboost-as-a-built-in-algo/output/


In [18]:
estimator = sagemaker.estimator.Estimator(image_uri = xgboost_container,
                                          hyperparameters = hyperparameters,
                                          role = sagemaker.get_execution_role(),
                                          instance_count = 1,
                                          instance_type = 'ml.m4.xlarge',
                                          volume_size = 5, #5 GB
                                          output_path = output_path,
                                          use_spot_instances = True,
                                          max_run = 300,
                                          max_wait = 600
                                         )

In [19]:
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type=content_type)

In [20]:
estimator.fit({'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-06-04-12-29-35-532


2024-06-04 12:29:35 Starting - Starting the training job...
2024-06-04 12:30:04 Starting - Preparing the instances for training......
2024-06-04 12:30:44 Downloading - Downloading input data...
2024-06-04 12:31:15 Downloading - Downloading the training image.....[2024-06-04 12:32:19.146 ip-10-0-152-138.us-west-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-04:12:32:19:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-04:12:32:19:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-06-04:12:32:19:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-04:12:32:19:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-06-04:12:32:19:INFO] Determined delimiter of CSV input is ','
[2024-06-04:12:32:19:INFO] Determined delimiter of CSV input is ','
[2024-06-04:12:32:19:INFO] Determined delimiter of CSV input is ','
[2024-06-04:12:32:19:INFO] Determined delimiter of CSV

In [21]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = estimator.deploy(initial_instance_count = 1, instance_type='ml.m4.xlarge', serializer=CSVSerializer())


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-06-09-55-48-733
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-06-06-09-55-48-733
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-06-06-09-55-48-733


------!

In [22]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-06-06-09-55-48-733'

In [23]:
#initialize parameters
start_date = datetime(2023, 1, 4)
end_date = datetime(2023, 1, 5)

#get the data
df_data = yf.download('AAPL', start = start_date, end = end_date)
df_data.reset_index(inplace=True)
df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-04,126.889999,128.660004,125.080002,126.360001,125.327507,89113600


In [25]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace=True)

data_features_array = df_data.values
data_features_array

array([[1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02,
        8.91136000e+07]])

In [27]:
Y_pred_Fcn = xgb_predictor.predict(data_features_array).decode('utf-8')
print(Y_pred_Fcn, type(Y_pred_Fcn))

'128.92034912109375\n'

In [29]:
from sagemaker.serializers import CSVSerializer

Serialized_Input_Fcn = CSVSerializer().serialize([[1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02,
        8.91136000e+07]])

print(Serialized_Input_Fcn, type(Serialized_Input_Fcn))

Y_pred_Fcn = xgb_predictor.predict(data_features_array).decode('utf-8')
print(Y_pred_Fcn, type(Y_pred_Fcn))

126.889999,128.660004,125.080002,126.360001,89113600.0 <class 'str'>
128.92034912109375
 <class 'str'>


In [31]:
Input = [[1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07], 
        [1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07], 
        [1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07]]
Serialized_Input = ', '.join(map(str, Input[0]))

print(Serialized_Input, type(Serialized_Input))

Y_pred = xgb_predictor.predict(Serialized_Input).decode('utf-8')
Y_pred

126.889999, 128.660004, 125.080002, 126.360001, 89113600.0 <class 'str'>


'128.92034912109375\n'

In [35]:
import boto3

ENDPOINT_NAME = 'sagemaker-xgboost-2024-06-06-09-55-48-733'
runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    inputs = event['data']
    
    result = []
    
    for input in inputs:
        serialized_input = ', '.join(map(str, input))
    
        response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                                     ContentType = 'text/csv',
                                     Body = serialized_input)
    
        result.append(response['Body'].read().decode())
    
    return result

['128.92034912109375\n', '128.92034912109375\n', '128.92034912109375\n']

In [ ]:
Input_json = { "data":
        [[1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07], 
        [1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07], 
        [1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07]
        ]
}

result = lambda_handler(Input_json, __)
result

In [36]:
#importing the requests library
import requests

#defining the api-endpoint
API_ENDPOINT = "https://zqt11le4w9.execute-api.us-west-1.amazonaws.com/xgbmodel"

#data to be sent to api
json = {"data":[[1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07], 
        [1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07], 
        [1.26889999e+02, 1.28660004e+02, 1.25080002e+02, 1.26360001e+02, 8.91136000e+07]]
       }

#sending post request and saving response as response object
r = requests.post(url = API_ENDPOINT, json = json)


In [39]:
print(f"Status Code: {r.status_code}, Response: {r.json()}")

Status Code: 200, Response: ['128.92034912109375\n', '128.92034912109375\n', '128.92034912109375\n']
